# Transfer Learning

You’ve used a pre-trained model to make predictions. That gives you great results if you want to classify images into the categories used by the original models. But what if you have a new use-case, and you don’t categorize images in exactly the same way as the categories for the pre-trained model?

For example, I might want a model that can tell if a photo was taken in an urban area or a rural area. My pre-trained model doesn’t classify images into those two specific categories.  We could build a new model from scratch for this specific purpose. But to get good results, we’d need thousands of photos with labels for which are urban and which are rural.

A method called transfer learning will give us good results with far less data. Transfer learning takes what a model learned while solving one problem, and applies it to a new application.

Remember that early layers of a deep learning model identify simple shapes. Later layers identify more complex visual patterns, and the very last layer makes predictions. Most layers from a pre-trained are useful in new applications, because most computer vision problems involve similar low-level visual patterns.  We’ll re-use most of the pre-trained ResNet model, and just replace the final layer that made predictions.

<img src="https://i.imgur.com/NdcEZhf.png" width=500px>

Some layers before that in the pretrained model may identify features like roads, buildings, windows, open fields, etc. We’ll drop in a replacement for the last layer of the ResNet model. This new last layer will predict whether an image is rural or urban based on the results of the previous layer.

The last layer of what’s left has information about our photo contents stored as a series of numbers in a tensor. It should be a 1-dimensional tensor, which is also called a vector. The vector can be shown as a series of dots. Each dot is called a node. The first node represents the first number in the vector. The second node represents the second number. And so on. Practical models have more nodes than we’ve drawn here.  

<img src="https://i.imgur.com/ZsJWiDV.png" width=350px>

We want to classify the images into two categories, urban and rural. After the last layer we keep of the pre-trained model, we add a new layer with two nodes. One node to capture how urban the photo is, and another to capture how rural it is. In theory, any node in the last layer before prediction might inform how urban it is. So the urban measure can depend on all the nodes in this layer.  We draw connections to show that possible relationship.  For the same reason, the information at each node might affect our measure of how rural the photo is.

We have a lot of connections here, and we’ll use training data to determine which nodes suggest an image is urban, which suggest it is rural, and which don’t matter.  That is, we’re going to the training the last layer model. In practice, that training data will be photos that are labeled as being either rural or urban.  We’ll cover more mathematical detail on this training step in a later lesson.

Notice that we allow all features from one layer to influence or be connected with the prediction layer.  When this happens, we describe the last layer as being a dense layer.  

One other note: When classifying something into only 2 categories, we could get by with only one node at the output.  In this case, a prediction for how urban a photo is would also be a measure of how rural it is.  If a photo is 80% likely to be urban, it would be 20% likely to be rural. But we’ve kept two separate nodes at the output layer.  Using a separate node for each possible category in the output layer will help us transition to cases when we want to predict with more than 2 categories.

In both the current case and the case with more categories, we’ll get a score for each category, then apply a function called softmax. The softmax function will transform the scores to probabilities. So, they’ll all be positive, and they will sum to 1.

<img src="https://i.imgur.com/RutdkVs.png" width=500px>

We could then work with those probabilities however we want. Let’s see how we do all this in code.

# Sample Code

### Specify Model

We’ll introduce two new classes from TensorFlow.  First is `Sequential`.  This is just saying we’re going to have a model that’s a sequence of layers, one after the other.  There are some exotic models that don’t fit this structure. For now, all models we'll cover sequential. We’ll also want to add a `Dense` layer to use as our new prediction layer.

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D

num_classes = 2
resnet_weights_path = '../input/resnet50/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'

my_new_model = Sequential()
my_new_model.add(ResNet50(include_top=False, pooling='avg', weights=resnet_weights_path))
my_new_model.add(Dense(num_classes, activation='softmax'))

# Say not to train first layer (ResNet) model. It is already trained
my_new_model.layers[0].trainable = False

In this application, we classify photos into 2 categories or classes, urban and rural. We’ll save that as `num_classes`.

Next we build the model. We set up a sequential model that we can add layers to. First we add a pretrained ResNet50 model. When creating the ResNet model, we’ve written `include_top=False`. This is how we specify that we want to exclude the last layer of the ResNet model that makes predictions into the thousand ImageNet categories.  We’ll also use a file that doesn’t include the weights for that layer.

The argument `pooling=average` says that if we had extra channels in our tensor at the end of this step, we want to collapse them to a 1D tensor by taking an average.  We’ll come back to intricacies of pooling in a later lesson. But now we have a pretrained model that creates the layer you saw in the graphic.  We’ll add a Dense layer to make predictions.  We specify the number of nodes in this layer, which in this case is the number of classes. Then we say we want to apply the softmax function to turn it into probabilities.

Finally, we’ll tell TensorFlow not to train the first layer of the sequential model, the ResNet50 layers. This is because that’s the model that was already pre-trained with the ImageNet data. 

### Compile Model

Now we’ll get to a more complex line of code, the compile command.

I’ll describe the broad concept here and we’ll give a more complete explanation of the underlying theory in a later lesson.  The compile command tells TensorFlow how to update the relationships in the Dense connections when we are doing the training with our data.  

In [ ]:
my_new_model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

We have a measure of loss or inaccuracy we want to minimize. We specify it as `categorical_crossentropy`. If you are familiar with log-loss, this is another term for the same thing.

We use an algorithm called stochastic gradient descent to minimize the categorical cross-entropy loss. We asked it to report the accuracy metric, the fraction of correct predictions. This is easier to interpret than categorical cross-entropy scores, so it’s nice to print it out and see how the model is doing.

### Loading the Image Data

Our raw data is broken into a directory of training data and a directory of validation data. Within each of those, we have one subdirectory for the urban pictures and another for the rural pictures. TensorFlow provides a great tool for working with images grouped into directories by their label.  This is the `ImageDataGenerator`. 

There are two steps to using `ImageDataGenerator`. First we create the generator object in the abstract. I’ll tell it that we want to apply the ResNet preprocessing function every time it reads in an image. You used this function before, to be consistent with how the ResNet model was created.

In [ ]:
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

image_size = 224
data_generator = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = data_generator.flow_from_directory(
        '../input/urban-and-rural-photos/train',
        target_size=(image_size, image_size),
        batch_size=24,
        class_mode='categorical')

validation_generator = data_generator.flow_from_directory(
        '../input/urban-and-rural-photos/val',
        target_size=(image_size, image_size),
        class_mode='categorical')

Then we use this `flow_from_directory` command. We tell it what directory that data is in, what size image we want, how many images to read in at a time (the batch size), and we tell it we’re classifying data into different categories. We do the same thing to set up a way to read the validation data.

The `ImageDataGenerator` is especially very valuable when working with large datasets, because we don’t need to hold the whole dataset in memory at once. But it’s nice here, even with a small dataset. Note that these are generators which means we need to iterate over them to get data out.

### Fitting the Model

Now we fit the model.  We tell it the training data comes from `train_generator`.  We said to read 24 images at a time, and we have 72 training images. So we’ll go through three steps of twelve images.  Then we say the validation data comes from validation_generator. Validation generator read 20 images at a time, and we have 20 images of validation data.  So we can use 1 step.

As the model training is running, we’ll see progress updates showing with our loss function and the accuracy. It updates the connections in the dense layer, that is the model’s impression of what makes an urban photo and what makes a rural photo, and it makes those updates in 3 steps. When it’s done, it got 79% of the training data right.  Then it examines the validation data. It gets 95% of those right. 19 out of 20.

I should mention that this is a really small dataset and you should be hesitant about relying on validation scores from such a small amount of data.  We’re starting with small datasets so you can get some experience under your belt with models that can be trained quickly.

In [ ]:
my_new_model.fit_generator(
        train_generator,
        steps_per_epoch=3,
        validation_data=validation_generator,
        validation_steps=1)

Even with the small training dataset, this accuracy score is really good. We trained on 72 photos. You could easily take that many photos on your phone, upload them to Kaggle, and build a very accurate model to distinguish almost anything you care about. I think that’s pretty cool.

This may feel like a lot of new ideas for you to take in. Here’s the plan. We have one exercise for you to build a model yourself using transfer learning. 

After you have hands-on experience with transfer learning, I’ll show you a simple but powerful trick called data augmentation. Data augmentation easily improves your computer vision models when working with small and medium sized datasets.

Then I’ll come back and explain some of the theory that makes all of this possible.  That will clarify some loose ends and set the stage for you to build models from scratch.

### Note on Results:
The printed validation accuracy can be meaningfully better than the training accuracy at this stage. This can be puzzling at first.

It occurs because the training accuracy was calculated at multiple points as the network was improving (the numbers in the convolutions were being updated to make the model more accurate).  The network was inaccurate when the model saw the first training images, since the weights hadn't been trained/improved much yet.  Those first training results were averaged into the measure above.

The validation loss and accuracy measures were calculated **after** the model had gone through all the data.  So the network had been fully trained when these scores were calculated.

This isn't a serious issue in practice, and we tend not to worry about it.

# Your Turn
**[Try transfer learning](#$EXERCISE_FORKING_URL$)** yourself.
